In [1]:
import requests
import requests_ftp
import requests_cache
import lxml
from lxml import html
from bs4 import BeautifulSoup
from collections import Counter
from matplotlib import pyplot as plt
import pandas as pd
import urllib
import json
import cloudscraper
import sqlalchemy as sqla

In [2]:
engine = sqla.create_engine('sqlite:///top200data')
url = "https://spotifycharts.com/regional/global/daily/2021-05-25"
scraper = cloudscraper.create_scraper()
r = scraper.get("https://spotifycharts.com/regional/global/daily/2021-05-25")
clim = BeautifulSoup(r.text,'lxml')

In [80]:
regiondata=pd.DataFrame(columns=['code','country'])

for i in range(70):
    regiondata.loc[i,"code"]=clim.find_all(name='li')[:70][i].get('data-value')
    regiondata.loc[i,"country"]=clim.find_all(name='li')[:70][i].text

pd.set_option("max_rows", None)
regiondata=regiondata.loc[[1,4,9,10,11,13,14,15,20,21,28,30,44,46,50,51,55,62,67]]
regiondata["region"]=["NA", "SA", "SA", "SA","NA", "SA" , "SA" , "NA" ,  "NA" , "SA" ,  "NA" , "NA" , "NA" , "NA" ,  "NA" , "SA" ,  "SA" ,  "NA" , "SA"]

regiondata=regiondata.reset_index()
pd.reset_option('^display.', silent=True)
regiondata

,index,code,country,region
0,1,us,United States,NA
1,4,ar,Argentina,SA
2,9,bo,Bolivia,SA
3,10,br,Brazil,SA
4,11,ca,Canada,NA
5,13,cl,Chile,SA
6,14,co,Colombia,SA
7,15,cr,Costa Rica,NA
8,20,do,Dominican Republic,NA
9,21,ec,Ecuador,SA


In [20]:
url = "https://spotifycharts.com/regional/{}/weekly/2021-05-14--2021-05-21".format(regiondata.loc[1,"code"])
url

'https://spotifycharts.com/regional/us/weekly/2021-05-14--2021-05-21'

In [25]:
rank = clim.find_all(name='td',attrs={'class':'chart-table-position'})
info = clim.find_all(name='td',attrs={'class':'chart-table-track'})
link = clim.find_all(name='a')[6:-5] #only song links
streams = clim.find_all(name='td',attrs={'class':'chart-table-streams'})
info[0].text.splitlines()[2][3:]
streams[0].text

'12,586,645'

In [18]:
rank

[<td class="chart-table-position">1</td>,
 <td class="chart-table-position">2</td>,
 <td class="chart-table-position">3</td>,
 <td class="chart-table-position">4</td>,
 <td class="chart-table-position">5</td>,
 <td class="chart-table-position">6</td>,
 <td class="chart-table-position">7</td>,
 <td class="chart-table-position">8</td>,
 <td class="chart-table-position">9</td>,
 <td class="chart-table-position">10</td>,
 <td class="chart-table-position">11</td>,
 <td class="chart-table-position">12</td>,
 <td class="chart-table-position">13</td>,
 <td class="chart-table-position">14</td>,
 <td class="chart-table-position">15</td>,
 <td class="chart-table-position">16</td>,
 <td class="chart-table-position">17</td>,
 <td class="chart-table-position">18</td>,
 <td class="chart-table-position">19</td>,
 <td class="chart-table-position">20</td>,
 <td class="chart-table-position">21</td>,
 <td class="chart-table-position">22</td>,
 <td class="chart-table-position">23</td>,
 <td class="chart-ta

In [7]:
info[0].text.splitlines()


['', 'good 4 u', 'by Olivia Rodrigo']

In [8]:
id = "date_{}_region".format(int(rank[0].text))
filler = (id, int(rank[0].text), info[0].text.splitlines()[1] , "date" , info[0].text.splitlines()[2][3:], "region")
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'region VARCHAR, '
               'PRIMARY KEY (id));')
engine.execute('INSERT INTO "top200" '
               '(id, rank, song, date, artist,region) '
               'VALUES (?,?,?,?,?,?) ', filler )
pd.read_sql('select * from top200',engine)

,id,rank,song,date,artist,region
0,date_1_region,1,good 4 u,date,Olivia Rodrigo,region


In [92]:
import time
start_time = time.time()
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date DATE, '
               'artist VARCHAR, '
               'region VARCHAR, '
               'songcode VARCHAR, '
               'streams INTEGER NOT NULL, '
               'PRIMARY KEY (id));')


for i in range(200):
    id = "date_{}_region".format(int(rank[i].text))
    filler = (id, int(rank[i].text), info[i].text.splitlines()[1] , "date" , info[i].text.splitlines()[2][3:], "region", link[i].get('href')[-22:],streams[i].text)

    engine.execute('INSERT INTO "top200" '
                   '(id, rank, song, date, artist,region, songcode, streams) '
                   'VALUES (?,?,?,?,?,?,?,?) ', filler )
print("--- %s seconds ---" % (time.time() - start_time))
pd.read_sql('select * from top200',engine)


--- 3.062173843383789 seconds ---


,id,rank,song,date,artist,region,songcode,streams
0,date_1_region,1,Chantaje (feat. Maluma),date,Shakira,region,6mICuAdrwEjh6Y6lroV2Kg,"1,478,972"
1,date_2_region,2,Reggaetón Lento (Bailemos),date,CNCO,region,3AEZUABDXNtecAOSC1qTfo,"1,305,366"
2,date_3_region,3,Vente Pa' Ca (feat. Maluma),date,Ricky Martin,region,7DM4BPaS7uofFul3ywMe46,"1,160,857"
3,date_4_region,4,Safari,date,"J Balvin, Pharrell Williams, BIA, Sky",region,6rQSrBHf7HlZjtcMZ4S4bO,"975,009"
4,date_5_region,5,Cuando Se Pone a Bailar,date,Rombai,region,1MpKZi1zTXpERKwxmOu1PH,"912,763"
...,...,...,...,...,...,...,...,...
195,date_196_region,196,Playa y Arena (Remix),date,"Mark B., Gabriel Pagan, Ozuna",region,4Q6Z7fn2xVhPZ1xn0XfEOY,"86,684"
196,date_197_region,197,Aventura,date,"Pijama Party, Valentina Olguin",region,1y9mv4NMheIuIUKf9hsIsy,"85,948"
197,date_198_region,198,Trátame Suavemente - Remasterizado 2007,date,Soda Stereo,region,65DBZofI0b79kfHTcWWDuU,"85,933"
198,date_199_region,199,Su Novio Anterior,date,"El Villano, J-One, Newyorkeeno",region,64CvAy6EPQpxCNbW2LNze9,"85,822"


In [78]:
start_time = time.time()
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'regioncode VARCHAR, '
               'songcode VARCHAR, '
               'streams INTEGER NOT NULL, '
               'PRIMARY KEY (id));')

scraper = cloudscraper.create_scraper()
for place in range(19):


    url = "https://spotifycharts.com/regional/{}/weekly/2020-12-25--2021-01-01".format(regiondata.loc[place,'code'])

    r = scraper.get(url)
    clim = BeautifulSoup(r.text,'lxml')
    
    rank = clim.find_all(name='td',attrs={'class':'chart-table-position'})
    info = clim.find_all(name='td',attrs={'class':'chart-table-track'})
    link = clim.find_all(name='a')[6:-5]  #only song links
    streams = clim.find_all(name='td',attrs={'class':'chart-table-streams'})
    
    for i in range(len(rank)):
        id = "date_{}_{}".format(int(rank[i].text),place)
        filler = (id, int(rank[i].text), info[i].text.splitlines()[1] , "date" , info[i].text.splitlines()[2][3:], regiondata.loc[place,'code'], link[i].get('href')[-22:], streams[i].text)

        engine.execute('INSERT INTO "top200" '
                       '(id, rank, song, date, artist, regioncode, songcode, streams) '
                   'VALUES (?,?,?,?,?,?,?,?) ', filler )


print("--- %s seconds ---" % (time.time() - start_time))
#pd.read_sql('select * from top200',engine)


--- 86.46330976486206 seconds ---


In [81]:
pd.reset_option('^display.', silent=True)
pd.read_sql('select * from top200',engine)


,id,rank,song,date,artist,region,regioncode,songcode,streams
0,date_1_0,1,Rockin' Around The Christmas Tree,date,Brenda Lee,None,us,2EjXfH91m7f8HiJN1yQg97,"5,662,961"
1,date_2_0,2,34+35,date,Ariana Grande,None,us,6Im9k8u9iIzKMrmV7BWtlF,"5,425,062"
2,date_3_0,3,Jingle Bell Rock,date,Bobby Helms,None,us,7vQbuQcyTflfCIOu3Uzzya,"5,326,633"
3,date_4_0,4,Mood (feat. iann dior),date,24kGoldn,None,us,3tjFYV6RSFtuktYl3ZtYcq,"5,204,578"
4,date_5_0,5,"Lemonade (feat. Gunna, Don Toliver & NAV)",date,Internet Money,None,us,7hxHWCCAIIxFLCzvDgnQHX,"5,165,110"
...,...,...,...,...,...,...,...,...,...
3795,date_196_18,196,BOOKER T,date,Bad Bunny,None,uy,26w9NTiE9NGjW1ZvIOd1So,"13,961"
3796,date_197_18,197,Bajito No,date,Los Negroni,None,uy,2NbMHtHIp8jH4oglpEx0Jc,"13,873"
3797,date_198_18,198,Levitating (feat. DaBaby),date,Dua Lipa,None,uy,5nujrmhLynf4yMoMtj8AQF,"13,858"
3798,date_199_18,199,AYNEA REMIX,date,"FMK, Maria Becerra, Beret",None,uy,5dDxdj2MrqrYCGzqXfJycL,"13,844"


In [142]:
import datetime
start_date = datetime.date(2017, 1, 6)
end_date = datetime.date(2021, 1, 1)
delta = datetime.timedelta(days = 7)
ls_date = []
while start_date <= end_date:
    ls_date.append(str(start_date))
    start_date += delta
ls_date[121]

'2019-05-03'

In [143]:
import time
start_time = time.time()
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'regioncode VARCHAR, '
               'songcode VARCHAR, '
               'streams INTEGER NOT NULL, '
               'PRIMARY KEY (id));')

scraper = cloudscraper.create_scraper()

for k in range(121,207):
    for place in range(len(regiondata)):

        url = "https://spotifycharts.com/regional/{}/weekly/{}--{}".format(regiondata.loc[place,'code'], ls_date[k], ls_date[k+1])
        r = scraper.get(url)
        clim = BeautifulSoup(r.text,'lxml')
        rank = clim.find_all(name='td',attrs={'class':'chart-table-position'})
        info = clim.find_all(name='td',attrs={'class':'chart-table-track'})
        link = clim.find_all(name='a')[6:-5] #only song links
        streams = clim.find_all(name='td',attrs={'class':'chart-table-streams'})
          

        for i in range(len(rank)):
            id = "{}_{}_{}".format(ls_date[k],int(rank[i].text),regiondata.loc[place,'code'])
            filler = (id, int(rank[i].text), info[i].text.splitlines()[1] , ls_date[k] , info[i].text.splitlines()[2][3:], regiondata.loc[place,'code'], link[i].get('href')[-22:], streams[i].text)
            engine.execute('INSERT INTO "top200" '
                           '(id, rank, song, date, artist, regioncode, songcode, streams) '
                       'VALUES (?,?,?,?,?,?,?,?) ', filler )
    

print("--- %s seconds ---" % (time.time() - start_time))

# weeklytop200_df =  pd.read_sql('select * from top200',engine)

pd.read_sql('select * from top200',engine)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [145]:
#pd.read_sql('SELECT * FROM top200 WHERE "id" LIKE "%_1_%"; ', engine)
pd.read_sql('select * from top200',engine).to_csv("top200_190503--200522.csv")

In [30]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='3b8a7beb4a434db98620cfae1983cffc',client_secret='a5bc3f14c035496690b540b81c5053f5',))

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])


Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [18]:
uniquesongs=pd.read_sql('SELECT DISTINCT song,songcode FROM top200',engine)
uniquesongs

,song,songcode
0,good 4 u,4ZtFanR9U6ndgddUvNcjcG
1,Butter,3VqeTFIvhxu3DIe4eZVzGq
2,deja vu,6HU7h9RYOaPRFeh0R3UeAr
3,MONTERO (Call Me By Your Name),67BtfxlNbhBmCDR2L2l8qd
4,Kiss Me More (feat. SZA),748mdHapucXQri7IAO8yFK
...,...,...
195,Savage Love (Laxed - Siren Beat),1xQ6trAsedVPCdbtDAmk0c
196,La Nota,5VzNsUnLmYcRfRfxpQsusa
197,If the World Was Ending - feat. Julia Michaels,2kJwzbxV2ppxnQoYw4GLBZ
198,Somebody That I Used To Know,1qDrWA6lyx8cLECdZE7TV7


In [73]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='3b8a7beb4a434db98620cfae1983cffc',client_secret='a5bc3f14c035496690b540b81c5053f5',))

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])


Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [38]:
results = spotify.search(q='artist:' + 'Doja Cat', type='artist')
items = results['artists']['items']
items['items']

TypeError: list indices must be integers or slices, not str

In [27]:
url="https://www.chosic.com/music-genre-finder"
r=requests.get(url, params={'track':'4ZtFanR9U6ndgddUvNcjcG'})
glimhtml = r.text
glim = BeautifulSoup(glimhtml,'lxml')
#glim.findall(name = 'div', attrs = {'class':'spotify_result'})


In [56]:
(pd.read_sql('SELECT DISTINCT artist FROM top200',engine)['artist']).apply(lambda x: x.split(', ')[0])

0        Olivia Rodrigo
1                   BTS
2             Lil Nas X
3              Doja Cat
4         Justin Bieber
             ...       
135           Jawsh 685
136       Manuel Turizo
137             JP Saxe
138               Gotye
139    The Chainsmokers
Name: artist, Length: 140, dtype: object

In [76]:
spotify.audio_analysis('4ZtFanR9U6ndgddUvNcjcG')

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1621579276,
  'analysis_time': 7.92829,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 3928134,
  'duration': 178.14667,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.22744,
  'start_of_fade_out': 175.50803,
  'loudness': -5.044,
  'tempo': 166.928,
  'tempo_confidence': 0.206,
  'time_signature': 4,
  'time_signature_confidence': 0.994,
  'key': 9,
  'key_confidence': 0.108,
  'mode': 1,
  'mode_confidence': 0.317,
  'codestring': 'eJxVmgeyHTkOBK_SR6A397_YZhafpNmYiD8iXzcNTKEAdJ27z3H6_Mq3Rx13nXu-etvX-6i7trO-udd3Wq2zrf3ddr7Fn9XK-uoYhQf7rCxReO2Mr_J4bYVVevHXxoNrT59j1VrWObWPxQabdfc6q9R-5zfa4ZW521qj7W-cwoKlrHvqme2bfYyPpWbhV840-2LI422u8s3Dy3uO2nbhudXX-Wa7rZ4917d8pM_J8cq-H6fxIL3VwVY8fXll7F1WHfN-u5bx9VFa6bV9u102qpdDLG7Mqyy1ym1DIe01znf3Wbuf3